In [23]:
import json
import pandas as pd
from pathlib import Path

# Fonction pour lire les données de transcription
def read_transcription(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Fonction pour lire les données du graphe de discours
def read_discourse_graph(file_path):
    with open(file_path, 'r') as file:
        data = [line.strip().split() for line in file]

    # Convertir les valeurs non numériques en indices numériques
    data = [(int(start), relation, int(end)) if start.isdigit() and end.isdigit() else (start, relation, end) for start, relation, end in data]

    return data

def create_dataframe(dialogue_id, transcription, discourse_graph, relation_dict,speaker_dict ):
    rows = []

    # Parcourir toutes les arêtes du graphe de discours
    for edge in discourse_graph:
        index_start, relation_type, index_end = edge

        # Récupérer les informations du locuteur
        speaker = transcription[index_start]['speaker']

        # Convertir le type de relation en entier en utilisant le dictionnaire
        speaker_id = speaker_dict.get(speaker, -1)

        # Récupérer la phrase
        text = transcription[index_start]['text']

        # Convertir le type de relation en entier en utilisant le dictionnaire
        relation_type_id = relation_dict.get(relation_type, -1)

        # Ajouter une ligne au DataFrame
        rows.append({
            'dialogue_id': dialogue_id,
            'index_start': index_start,
            'text' : text,
            'index_end': index_end,
            'speaker_type': speaker_id,
            'speaker_text' : speaker,
            'relation_type': relation_type_id,
            'relation_text' : relation_type
        })

    # Créer le DataFrame
    df = pd.DataFrame(rows)

    return df


# Fonction pour créer le dictionnaire de conversion des relations
def create_relation_dict(discourse_graph):
    relation_set = set()

    # Collecter toutes les relations uniques
    for edge in discourse_graph:
        relation_set.add(edge[1])

    # Créer un dictionnaire de conversion
    relation_dict = {relation: idx for idx, relation in enumerate(relation_set)}

    return relation_dict

# Fonction pour créer le dictionnaire de conversion des speakers
def create_speaker_dict(transcription):
    speaker_set = set()

    # Collecter tous les locuteurs uniques
    for utterance in transcription:
        speaker_set.add(utterance['speaker'])

    # Créer un dictionnaire de conversion
    speaker_dict = {speaker: idx for idx, speaker in enumerate(speaker_set)}

    return speaker_dict

# Remplacez 'votre_chemin' par le chemin correct
chemin_dossier = Path("data/training")

# Liste pour stocker les DataFrames de chaque dialogue
dfs = []

def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]

# Remplacez 'vos_dialogue_ids' par votre liste réelle d'identifiants de dialogue
dialogue_ids = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
dialogue_ids = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in dialogue_ids])
dialogue_ids.remove('IS1002a')
dialogue_ids.remove('IS1005d')
dialogue_ids.remove('TS3012c')

# Liste pour stocker les DataFrames de chaque dialogue
dfs = []

# Parcourir chaque dialogue
for dialogue_id in dialogue_ids:
    # Lire les données de transcription et de graphe de discours
    transcription = read_transcription(chemin_dossier / f'{dialogue_id}.json')
    discourse_graph = read_discourse_graph(chemin_dossier / f'{dialogue_id}.txt')

    # Créer le dictionnaire de conversion des relations
    relation_dict = create_relation_dict(discourse_graph)
    speaker_dict = create_speaker_dict(transcription)

    # Créer le DataFrame pour le dialogue actuel
    df_dialogue = create_dataframe(dialogue_id, transcription, discourse_graph, relation_dict, speaker_dict)

    # Ajouter le DataFrame à la liste
    dfs.append(df_dialogue)

# Concaténer tous les DataFrames en un seul
df_final = pd.concat(dfs, ignore_index=True)

# Afficher le DataFrame final
df_final.info()
df_final.head(25)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72526 entries, 0 to 72525
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   dialogue_id    72526 non-null  object
 1   index_start    72526 non-null  int64 
 2   text           72526 non-null  object
 3   index_end      72526 non-null  int64 
 4   speaker_type   72526 non-null  int64 
 5   speaker_text   72526 non-null  object
 6   relation_type  72526 non-null  int64 
 7   relation_text  72526 non-null  object
dtypes: int64(4), object(4)
memory usage: 4.4+ MB


,dialogue_id,index_start,text,index_end,speaker_type,speaker_text,relation_type,relation_text
0,ES2002a,0,Okay,1,0,PM,12,Continuation
1,ES2002a,1,Right,2,0,PM,12,Continuation
2,ES2002a,2,<vocalsound> Um well this is the kick-off meet...,3,0,PM,10,Explanation
3,ES2002a,3,Um <vocalsound> and um,4,0,PM,4,Elaboration
4,ES2002a,4,this is just what we're gonna be doing over th...,5,0,PM,12,Continuation
5,ES2002a,5,"Um so first of all , just to kind of make sure...",6,0,PM,4,Elaboration
6,ES2002a,6,I'm Laura and I'm the project manager .,7,0,PM,12,Continuation
7,ES2002a,7,<vocalsound> Do you want to introduce yourself...,8,0,PM,6,Acknowledgement
8,ES2002a,5,"Um so first of all , just to kind of make sure...",9,0,PM,12,Continuation
9,ES2002a,9,"Hi , I'm David and I'm supposed to be an indus...",10,3,ID,6,Acknowledgement
